My own notebook to play around with the [did](https://github.com/bcallaway11/did) package by Brantly Callaway and Pedro H.C. Sant’Anna. See the vignette [Getting Started with the did Package](https://cran.r-project.org/web/packages/did/vignettes/did-basics.html).

Still very much work in progress!

Notes
 - Used conda environment in the notebook is `dev2023a` from [here](https://github.com/vvoutilainen/dsenvs/blob/main/condaenv.md). *did* package was installed by:
   - First installing *r-ggpubr* to `dev2023a` via conda/mamba. This also downloaded packages *r-pbkrtest*, *r-car*, *r-rstatix*
   - Next, *did* was installed using R's `install.packages()` command with `dependencies=TRUE`. This also installed a few additional packages (*trust*, *BMisc*, *pbapply* , *DRDID*).
 - Repeated cross-section can be created by using `panel=FALSE` in function `build_sim_dataset`.
 - By default, the did package uses the group of units that never participate in the treatment as the control group. The "not yet treated" include the never treated as well as those units that, for a particular point in time, have not been treated yet (though they eventually become treated).
 
Questions

 - How is `G` in data used? Why does "never-treated" get `G=0`? I guess the point is that zeroeth period does not exist in the period column and this is why it is assigned as the "time of treatment" for the control group. Check! 

# Imports and auxiliary functions

In [ ]:
%%capture
import numpy as np
np.random.seed(1)
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import rpy2
%load_ext rpy2.ipython

from did_helpers import(
    simulate_did_data,
    plot_repcrossec_data,
    #plot_panel_data,
    parallel_trends_plot,
)

In [ ]:
%%R
library(did)
set.seed(1814)

In [ ]:
def plot_group_means(df):
    fig = plt.figure(figsize=(6, 3))
    ax = fig.add_subplot(1, 1, 1)
    _ = df.groupby([
        "G",
        "period",
    ]).agg({"Y": "mean"}).reset_index().pivot_table(
        values="Y",
        index="period",
        columns="G"
    ).plot(
        style='.--',
        ax=ax,
    )

def plot_group_cluster_means(df):
    fig = plt.figure(figsize=(8, 4))
    for counter, g in enumerate(df["G"].unique()):
        ax = fig.add_subplot(2, 2, counter+1)
        df.query("G=={}".format(g)).groupby([
            "cluster",
            "period",
        ]).agg({"Y": "mean"}).reset_index().pivot_table(
            values="Y",
            index="period",
            columns="cluster"
        ).plot(
            style='.--',
            legend=False,
            linewidth=0.5,
            markersize=2,
            ax=ax,
        )
        ax.set_title("G = {}".format(g))
    fig.tight_layout()

# Panel data

## Example with vanilla panel model: two-groups, two-periods

In [ ]:
%%R -o dta
sp = reset.sim(time.periods=2)
sp$thet = rep(0, sp$time.periods)
sp$theu = rep(0, sp$time.periods)
sp$bett = rep(0, sp$time.periods)
sp$betu = rep(0, sp$time.periods)
sp$te.bet.ind = rep(0, sp$time.periods)
sp$te.bet.X = rep(0, sp$time.periods)
sp$te.t = rep(0, sp$time.periods)
sp$te = 0
dta = build_sim_dataset(sp)

In [ ]:
print("Length of data frame".format(len(dta)))
print("Number of unique observations per id: {}".format(
    dta.groupby(["id"]).agg({"Y": "count"}).loc[:, "Y"].unique()
))
plot_group_means(dta)

In [ ]:
# Build model in statsmodels
dta["period_dummy"] = np.where(dta["period"] < 2, 0, 1)
dta["periodxtreatment_dummy"] = dta["period_dummy"] * dta["treat"]
reg_str = "Y ~ 1 + treat + period_dummy + periodxtreatment_dummy"
res = smf.ols(reg_str, data=dta).fit(
    cov_type="cluster",
    cov_kwds={
        "groups": dta["id"],
        "use_correction": False,
    }
)
print("Regression: {}\n".format(reg_str))
print(res.summary())

In [ ]:
%%R -i dta
# Repeat using did package, same result
# Warning relates to Wald pre-treatment period test 
out = att_gt(
    yname="Y",
    tname="period",,
    idname="id",
    gname="G",
    bstrap=F,
    panel=T,
    data=dta,
)
summary(out)

## Example from the vignette

In [ ]:
%%R -o dta,sp
sp = reset.sim()
time_periods = 4
sp$te.e <- 1:time_periods
dta = build_sim_dataset(sp)

In [ ]:
plot_group_means(dta)

In [ ]:
plot_group_cluster_means(dta)

In [ ]:
%%R
# Control group never treated
attgt_1 = att_gt(
    yname="Y",
    tname="period",
    idname="id",
    gname="G",
    xformla=~X,
    control_group="notyettreated",
    data=dta
)

# Control group never treated + not yet treated
attgt_2 = att_gt(
    yname="Y",
    tname="period",
    idname="id",
    gname="G",
    xformla=~X,
    control_group="nevertreated",
    data=dta
)

# Aggregate over groups
agg_simple_1 = aggte(attgt_1, type="simple")
agg_dynamic_1 = aggte(attgt_1, type="dynamic")
agg_group_1 = aggte(attgt_1, type="group")

agg_simple_2 = aggte(attgt_2, type="simple")
agg_dynamic_2 = aggte(attgt_2, type="dynamic")
agg_group_2 = aggte(attgt_2, type="group")

In [ ]:
%%R
summary(agg_simple_1)
summary(agg_simple_2)

In [ ]:
%%R
summary(attgt_2)
ggdid(attgt_2)

In [ ]:
%%R
summary(agg_dynamic_2)
ggdid(agg_dynamic_2)

In [ ]:
%%R
summary(agg_group_2)
ggdid(agg_group_2)

# Repeated cross section

## Example with *did* simulated wanilla two-group, two-period case

In [ ]:
%%R -o dta
sp = reset.sim(time.periods=2)
dta = build_sim_dataset(sp, panel=FALSE)
dta = subset(dta, select = -c(cluster) )

In [ ]:
print("Length of data frame".format(len(dta)))
print("Number of unique observations per id: {}".format(
    dta.groupby(["id"]).agg({"Y": "count"}).loc[:, "Y"].unique()
))
plot_group_means(dta)

In [ ]:
# Build model in statsmodels
dta["period_dummy"] = np.where(dta["period"] < 2, 0, 1)
dta["periodxtreatment_dummy"] = dta["period_dummy"] * dta["treat"]
reg_str = "Y ~ 1 + treat + period_dummy + periodxtreatment_dummy"
res = smf.ols(reg_str, data=dta).fit(
    cov_type="cluster",
    cov_kwds={
        "groups": dta["id"],
        "use_correction": False,
    }
)
print("Regression: {}\n".format(reg_str))
print(res.summary())

In [ ]:
%%R -i dta
# Repeat using did package, same result
# Warning relates to Wald pre-treatment period test 
out = att_gt(
    yname="Y",
    tname="period",
    gname="G",
    bstrap=F,
    panel=F,
    data=dta,
)
summary(out)

## Example with my own simulated multi-period, two-group case

In [ ]:
data_1 = simulate_did_data(
    param_datasettype="repeated cross-section",
    param_gamma_c=4,
    param_gamma_t=1,
)

In [ ]:
plot_repcrossec_data(data_1, figsize=(12, 8))

In [ ]:
parallel_trends_plot(data_1, figsize=(10, 6))

In [ ]:
def prepare_regression_frame(data):
    df = data["observed"].copy()

    # Code categories as dummies
    df["dummy_period"] = df["time_group"].map({
        "before": 0,
        "post": 1,
    })
    df["dummy_group"] = df["treatment_group"].map({
        "control": 0,
        "treatment": 1,
    })
    df["dummy_group_x_period"] = df["dummy_period"] * df["dummy_group"]

    # Time points as categorical/str
    df["t"] = df["t"].astype(str)

    return df

In [ ]:
df = prepare_regression_frame(data_1)
reg_str = "Y ~ -1 + dummy_group + t + dummy_group_x_period"
res = smf.ols(reg_str, data=df).fit()
print(res.summary())

In [ ]:
# Prepare for R-did
df["t"] = (df["t"].astype(int) + 1).astype("object")
df["t_num"] = df["t"].copy().astype(float)
df["tc_group_num"] = df["treatment_group"].copy().map({"control": 0, "treatment": 4})

In [ ]:
%%R -i df
out = att_gt(
    yname="Y",
    tname="t_num",
    gname="tc_group_num",
    bstrap=F,
    panel=F,
    data=df,
)
agg_simple = aggte(out, type="simple")
summary(out)
summary(agg_simple)